In [1]:
import keras
import h5py
import numpy as np
import matplotlib.pyplot as plt
from train_models import load_from_weights
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score
from generate_datasets import preprocessing_batch_data
import pandas as pd
import os

2024-06-21 13:01:17.780164: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 13:01:23.271990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-21 13:01:23.286053: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-21 13:01:23.286159: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUM

In [2]:
DATA_PATH = 'radiodataset0to16/'
TEST_FOLDER_PATH = DATA_PATH + "Test/"

SNR = 6

# filepath
ibcnn_filepath = 'ibcnn_best.keras'
sbcnn_filepath = 'sbcnn_best.keras'
student_filepath = 'student_best.keras'
student_scratch = 'student_scratch.keras'

paths = [
    sbcnn_filepath,
    ibcnn_filepath,
    student_filepath,
    student_scratch,
]

In [3]:
def get_data_from_path(snr:int):
    filepath = f'radiodataset0to16/Test/{snr}/test_data.h5'
    with h5py.File(filepath, 'r') as hf:
        x = hf['X'][:]
        y = hf['Y'][:]

    x = preprocessing_batch_data(x)
    y = np.argmax(y, axis=1)
    return x, y

# x, y = get_data_from_path(snr=SNR)

In [4]:
classes = ['OOK',
           '4ASK',
           '8ASK',
           'BPSK',
           'QPSK',
           '8PSK',
           '16PSK',
           '32PSK',
           '16APSK',
           '32APSK',
           '64APSK',
           '128APSK',
           '16QAM',
           '32QAM',
           '64QAM',
           '128QAM',
           '256QAM',
           'AM-SSB-WC',
           'AM-SSB-SC',
           'AM-DSB-WC',
           'AM-DSB-SC',
           'FM',
           'GMSK',
           'OQPSK']

student = keras.models.load_model(student_filepath)
student_alone = keras.models.load_model(student_scratch)
teacher = load_from_weights(ibcnn_filepath)
teacher.name = 'teacher'

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 38 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 218 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
models = [student]
classes = np.array(classes)

for m in models:
    logits = m.predict(x)
    y_pred = np.argmax(logits, axis=1)
    print(m.name)
    print(classification_report(y, y_pred, target_names=classes))
    cm = confusion_matrix(y, y_pred)
    data = [(r, c, cm[r,c]) for c in range(len(classes)) for r in range(len(classes))]
    df = pd.DataFrame(data)

    df.columns = ['a','b','c']
      
    # save the dataframe as a csv file 
    df.to_csv(f"confusion_{SNR}dB_{m.name}.csv")

def generate_confusion_matrix(y_test, y_pred):
    """ Generates a confusion_matrix plot based on the given values.
    Args:
        y_test (any): the resulting y_test of the function "train_test_split".
        y_pred (any): the resulting value of the function "predict".
    Returns:
        _.
    """
    #logger = logging.getLogger('ThreatTrekker')
    #logger.debug('Plotting confusion matrix')

    cm = confusion_matrix(y_test, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # plot the confusion matrix using seaborn
    sns.set(rc={'figure.figsize': (10, 6)})  # Size in inches
    sns.heatmap(cm_norm, annot=True, cmap='Blues', fmt='.1f')

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    # plt.savefig(PLOTS_PATH + 'Confusion Matrix')
    plt.show()

2024-06-21 12:40:55.614799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 12:41:06.357227: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-21 12:41:06.491680: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-21 12:41:06.491780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUM

 20/307 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step 

I0000 00:00:1718973675.208822     166 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


307/307 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step
student
              precision    recall  f1-score   support

         OOK       1.00      1.00      1.00       409
        4ASK       0.93      0.99      0.96       409
        8ASK       0.99      0.92      0.95       409
        BPSK       1.00      1.00      1.00       409
        QPSK       1.00      1.00      1.00       409
        8PSK       1.00      1.00      1.00       409
       16PSK       1.00      0.13      0.23       409
       32PSK       0.53      1.00      0.69       409
      16APSK       0.99      0.99      0.99       409
      32APSK       0.97      0.96      0.96       409
      64APSK       0.78      0.44      0.56       409
     128APSK       0.70      0.58      0.63       409
       16QAM       0.82      0.98      0.90       409
       32QAM       0.78      0.45      0.57       409
       64QAM       0.60      0.17      0.27       409
      128QAM       0.32      0.68      0.44       409
      256QAM       0.44      0.

In [2]:
 
def evaluate_model_on_test_set(model):
    score = {}
    time_logs = []
    for root, dir_names, file_names in os.walk(TEST_FOLDER_PATH):
        if file_names:
            current_db = os.path.basename(root)
            current_file = root + '/' + file_names[0]
            print(f'db: {current_db}')
            print('file: ' + current_file)

            with h5py.File(current_file, 'r') as hf:
                x = hf['X'][:]
                y = hf['Y'][:]

            x = preprocessing_batch_data(x)
            y = np.argmax(y, axis=1)

            logits = model.predict(x)
            y_pred = np.argmax(logits, axis=1)

            f1_socre = f1_score(y, y_pred, average='macro')

            score[int(current_db)] = f1_socre
            print(f'score: {f1_socre}')
    score = {i: score[i] for i in np.sort(list(score.keys()))}
    # return score, time_logs
    return score


In [3]:
score = evaluate_model_on_test_set(student)

db: 16
file: radiodataset0to16/Test/16/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.9216537549001086
db: 4
file: radiodataset0to16/Test/4/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.6822332833181975
db: 12
file: radiodataset0to16/Test/12/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.9153301304651701
db: 14
file: radiodataset0to16/Test/14/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.9182500669710009
db: 6
file: radiodataset0to16/Test/6/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.7830151097080567
db: 10
file: radiodataset0to16/Test/10/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.9002011985061382
db: 0
file: radiodataset0to16/Test/0/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.45656360579698796
db: 8
file: radiodataset0to16/Test/8/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.8610093286226791
db: 2
file: radiodataset0to16/Test/2/test_data.h5
307/3

In [4]:
score = evaluate_model_on_test_set(student_alone)

db: 16
file: radiodataset0to16/Test/16/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
score: 0.8916502560442415
db: 4
file: radiodataset0to16/Test/4/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.6851953530236757
db: 12
file: radiodataset0to16/Test/12/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.8873172841537111
db: 14
file: radiodataset0to16/Test/14/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.8812702793597262
db: 6
file: radiodataset0to16/Test/6/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.7923891473935746
db: 10
file: radiodataset0to16/Test/10/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.8733549631365154
db: 0
file: radiodataset0to16/Test/0/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.46022186787705116
db: 8
file: radiodataset0to16/Test/8/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
score: 0.8499662791665186
db: 2
file: radiodataset0to16/Test/2/test_data.h5
307/3

In [5]:
score = evaluate_model_on_test_set(teacher)

db: 16
file: radiodataset0to16/Test/16/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 16s 31ms/step
score: 0.9604379922116518
db: 4
file: radiodataset0to16/Test/4/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
score: 0.7639839486596468
db: 12
file: radiodataset0to16/Test/12/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
score: 0.9556192189429457
db: 14
file: radiodataset0to16/Test/14/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
score: 0.9542830840747397
db: 6
file: radiodataset0to16/Test/6/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
score: 0.8745135838491794
db: 10
file: radiodataset0to16/Test/10/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
score: 0.9476076960118718
db: 0
file: radiodataset0to16/Test/0/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
score: 0.5113207955846605
db: 8
file: radiodataset0to16/Test/8/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
score: 0.9274734279971545
db: 2
file: radiodataset0to16/Test/2/test_data.h5

In [5]:
 def classification_report_on_test_set(model):
    y_pred = []
    y_true = []
    for root, dir_names, file_names in os.walk(TEST_FOLDER_PATH):
        if file_names:
            current_db = os.path.basename(root)
            current_file = root + '/' + file_names[0]
            print(f'db: {current_db}')
            print('file: ' + current_file)

            with h5py.File(current_file, 'r') as hf:
                x = hf['X'][:]
                y = hf['Y'][:]

            x = preprocessing_batch_data(x)
            y = np.argmax(y, axis=1)
            
            logits = model.predict(x)
            y_ = np.argmax(logits, axis=1)

            y_pred = np.append(y_pred, y_)
            y_true = np.append(y_true, y)

    print(classification_report(y_true, y_pred, target_names=classes))
    return


In [6]:
classification_report_on_test_set(student)

db: 16
file: radiodataset0to16/Test/16/test_data.h5


I0000 00:00:1718974896.933362    1411 service.cc:145] XLA service 0x7ff9400049c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718974896.933489    1411 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-06-21 13:01:36.953800: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-21 13:01:37.061160: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


 15/307 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step

I0000 00:00:1718974898.518391    1411 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


307/307 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step
db: 4
file: radiodataset0to16/Test/4/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
db: 12
file: radiodataset0to16/Test/12/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 14
file: radiodataset0to16/Test/14/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 6
file: radiodataset0to16/Test/6/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 10
file: radiodataset0to16/Test/10/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 0
file: radiodataset0to16/Test/0/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 8
file: radiodataset0to16/Test/8/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 2
file: radiodataset0to16/Test/2/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
              precision    recall  f1-score   support

         OOK       1.00      1.00      1.00      3681
        4ASK       0.87      0.97      0.91      3681
        8ASK       0.96      0.85      0.90      3681
       

In [7]:
classification_report_on_test_set(student_alone)

db: 16
file: radiodataset0to16/Test/16/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
db: 4
file: radiodataset0to16/Test/4/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 12
file: radiodataset0to16/Test/12/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 14
file: radiodataset0to16/Test/14/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 6
file: radiodataset0to16/Test/6/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 10
file: radiodataset0to16/Test/10/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
db: 0
file: radiodataset0to16/Test/0/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
db: 8
file: radiodataset0to16/Test/8/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
db: 2
file: radiodataset0to16/Test/2/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
              precision    recall  f1-score   support

         OOK       1.00      1.00      1.00      3681
        4ASK       0.92      0.93      0.93      3681
        8A

In [8]:
classification_report_on_test_set(teacher)

db: 16
file: radiodataset0to16/Test/16/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step
db: 4
file: radiodataset0to16/Test/4/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
db: 12
file: radiodataset0to16/Test/12/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
db: 14
file: radiodataset0to16/Test/14/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
db: 6
file: radiodataset0to16/Test/6/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
db: 10
file: radiodataset0to16/Test/10/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
db: 0
file: radiodataset0to16/Test/0/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
db: 8
file: radiodataset0to16/Test/8/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
db: 2
file: radiodataset0to16/Test/2/test_data.h5
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
              precision    recall  f1-score   support

         OOK       1.00      1.00      1.00      3681
        4ASK       0.94      0.95      0.94      3681
 